In [1]:
import os,sys
import tensorflow as tf
# import importlib
sys.path.append('/home/ubuntu/hacking/projects/deep-mediation/nov2020')
import auxiliaryfunctions
# importlib.reload(auxiliaryfunctions)
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
from scipy.stats import zscore,norm
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os
import pandas as pd
import random as python_random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error,r2_score

# from datagenerator import dataGenerator
from keras import backend as K

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, SGD


Bad key savefig.frameon in file /home/ubuntu/anaconda3/envs/deep_structure/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /home/ubuntu/anaconda3/envs/deep_structure/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /home/ubuntu/anaconda3/envs/deep_structure/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filen

In [2]:
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

early_stopping = True
epochs = 1000
batch_size = 4
use_dynamic_LR=False
# optimizer='Adam'
algo = 'svr' #['shallow','deep','svr'] 
patience = 50
result_path = '/home/ubuntu/hacking/projects/deep-mediation/nov2020/results/simulation-3-results'
num_subs = [1000]
num_iters = 20
num_runs = 100
new_shape = [28,28]

In [3]:
def make_plot(z,m,d,iter):
    plt.figure(figsize=(20,5))
    plt.subplot(1, 4, 1)
    plot_distribution(z,m,labels=['z','m'])

    plt.subplot(1, 4, 2)
    plot_distribution(z,d,labels=['z','d'])

    plt.subplot(1, 4, 3)
    plot_scatter(z,d,labels= ['z','d'])

    plt.subplot(1, 4, 4)
    plot_scatter(z,m,labels= ['z','m'])
    # plt.savefig(os.path.join(result_path,'result'+ str(iter) +'.png'),dpi=300)
    plt.show()

def plot_distribution(z,m,labels):
    sns.distplot(z, hist=True, rug=True,label=labels[0]);
    sns.distplot(m, hist=True, rug=True,label=labels[1]);
    plt.legend()

def plot_scatter(y_test,y_pred,labels):
    r = np.corrcoef(y_pred,y_test)[0,1]
    r = round(r,3)
    plt.scatter(y_test,y_pred)
    plt.xlabel(labels[0])
    plt.ylabel(labels[1])
    plt.title('r=%s'%r)

def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('MSE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

def plot_error(ax,values,title,y_lab,num_iterations):
    ax.plot(values)
    ax.set_xlim([0,num_iterations])
    ax.set_xlabel('#iterations')
    ax.set_ylabel(y_lab)
    ax.set_title(title)

def create_empty_df(num_runs,num_iters):
    # Creates an empty dataFrame
    a = np.empty((num_runs,num_iters))
    a[:] = np.nan
    dataFrame = None
    parameters = ['alpha0', 'beta0','alpha','beta','gamma']
    for params in parameters:
        iter = ['iter_'+str(i) for i in range(num_iters)]
        pdindex = pd.MultiIndex.from_product([[params], iter],
                                             names=['parameters', 'runs']) 
        frame = pd.DataFrame(a, columns = pdindex,index = range(0,num_runs))
        dataFrame = pd.concat([dataFrame,frame],axis=1)
    return dataFrame

def simulate_mediation(df,M,params_df,model,algo,num_subs,batch_size,epochs,n,n_runs,iterations):
    """
    """
    early_stopping=True
    # Initialize z with some phi
    print("Using intial random model parameters")
#     if algo == 'svr':
#         z = svr.predict(M.flatten())
    z = model.predict(M) 
    try:
        z = np.concatenate(z)
    except:
        pass
    # plot_scatter(z,df.m,labels=['z','m'])
    
    for i in range(0,iterations):
        print('Starting iteration... %s'%(i+1))
        #Check for sign change
        if np.corrcoef(z,df.Y)[0,1] < 0 :
            z = z*(-1)
        # Check for scaling
        z= zscore(z)

        lm = smf.ols(formula='z ~ X', data=df).fit()
        alpha0 = lm.params.loc['Intercept']
        alph = lm.params.loc['X'] 

        lm = smf.ols(formula='Y ~ z + X', data=df).fit()
        beta0 = lm.params.Intercept 
        bet = lm.params.z
        gam = lm.params.X 
        resid_std = np.std(lm.resid)
        e = df.Y - beta0 - (df.X*gam)
        h = alpha0 + df.X*alph
        d = (((bet*e)+h)/((bet**2)+1))
#         d = zscore(d)

        if algo== 'svr':
            model = auxiliaryfunctions.create_svr_model(M,d)
            z = model.predict(M)
        else:
            adam = Adam(lr=0.001, decay=0.03)
            model.compile(loss='mean_absolute_error',optimizer=adam)
            if early_stopping:
                es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
                history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=[es],
                                  shuffle=True,validation_split = 0.3)
            else:
                history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,
                              shuffle=True,validation_split = 0.3)
            z = model.predict(M)
       
        params_df.loc[n_runs]['alpha0','iter_'+str(i)]=alpha0
        params_df.loc[n_runs]['beta0','iter_'+str(i)]=beta0
        params_df.loc[n_runs]['beta','iter_'+str(i)]=bet
        params_df.loc[n_runs]['gamma','iter_'+str(i)]=gam
        params_df.loc[n_runs]['alpha','iter_'+str(i)]=alph

        try:
            z = np.concatenate(z)
        except:
            pass
       
    return params_df,z


In [6]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

for n in num_subs:
    start = 95
    output_df = create_empty_df(num_runs,num_iters)
    for runs in range(start,num_runs):
        K.clear_session()
        print("Running simulation for image shape %s starting with %s runs"%(n,runs+1))
    # simulate the dataset
        df,mediator,z = auxiliaryfunctions.simulate_dataset(n,resize= True,
                                          new_shape=new_shape,algo=algo,visualize=False,alpha=0)
        # create a model 
        if algo == 'svr':
            model = auxiliaryfunctions.create_svr_model(mediator,df.m)
        elif algo== 'shallow':
            input_shape = (n[0],n[1]*4,1)
            model = auxiliaryfunctions.create_shallow_model2D(input_shape=input_shape)
        else:
            input_shape = (n[0],n[1]*4,1)
            model = auxiliaryfunctions.create_deep_model2D(input_shape=input_shape)

        params_df,z_final = simulate_mediation(df,mediator,output_df,
                                      model,algo,n,batch_size,epochs,
                                      n,runs,num_iters)
        params_df.to_excel(os.path.join(result_path,'simulation-3-num-subs-'+str(n)+'-'+algo+'-part-2.xlsx'))

Running simulation for image shape 1000 starting with 96 runs
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   24.9s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   26.0s finished


Using intial random model parameters
Starting iteration... 1
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   40.5s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   41.6s finished


Starting iteration... 2
Fitting 5 folds for each of 16 candidates, totalling 80 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   22.7s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   23.1s finished


Starting iteration... 19
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   21.1s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   21.7s finished


Starting iteration... 20
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 out of  80 | elapsed:   26.0s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   27.3s finished


In [ ]:
!nvidia-smi

In [ ]:
!kill -9 3740

In [5]:
params_df

parameters    alpha0                                                    \
runs          iter_0    iter_1    iter_2    iter_3    iter_4    iter_5   
0                NaN       NaN       NaN       NaN       NaN       NaN   
1                NaN       NaN       NaN       NaN       NaN       NaN   
2                NaN       NaN       NaN       NaN       NaN       NaN   
3                NaN       NaN       NaN       NaN       NaN       NaN   
4                NaN       NaN       NaN       NaN       NaN       NaN   
..               ...       ...       ...       ...       ...       ...   
95          0.000036  0.000039  0.000042  0.000045  0.000049  0.000052   
96          0.002881  0.002875  0.002939  0.003003  0.003068  0.003133   
97          0.000065  0.000179  0.000302  0.000425  0.000549  0.000671   
98         -0.000506 -0.000453 -0.000409 -0.000363 -0.000316 -0.000266   
99         -0.000458 -0.000453 -0.000459 -0.000464 -0.000470 -0.000475   

parameters                                          ...     gamma            \
runs          iter_6    iter_7    iter_8    iter_9  ...   iter_10   iter_11   
0                NaN       NaN       NaN       NaN  ...       NaN       NaN   
1                NaN       NaN       NaN       NaN  ...       NaN       NaN   
2                NaN       NaN       NaN       NaN  ...       NaN       NaN   
3                NaN       NaN       NaN       NaN  ...       NaN       NaN   
4                NaN       NaN       NaN       NaN  ...       NaN       NaN   
..               ...       ...       ...       ...  ...       ...       ...   
95          0.000056  0.000060  0.000064  0.000068  ...  4.869206  4.862192   
96          0.003199  0.003266  0.003333  0.003400  ...  5.031911  5.035889   
97          0.000795  0.000916  0.001036  0.001155  ...  5.043360  5.050927   
98         -0.000214 -0.000161 -0.000104 -0.000046  ...  5.047047  5.061156   
99         -0.000480 -0.000486 -0.000491 -0.000496  ...  5.009631  5.008846   

parameters                                                              \
runs         iter_12   iter_13   iter_14   iter_15   iter_16   iter_17   
0                NaN       NaN       NaN       NaN       NaN       NaN   
1                NaN       NaN       NaN       NaN       NaN       NaN   
2                NaN       NaN       NaN       NaN       NaN       NaN   
3                NaN       NaN       NaN       NaN       NaN       NaN   
4                NaN       NaN       NaN       NaN       NaN       NaN   
..               ...       ...       ...       ...       ...       ...   
95          4.855003  4.847582  4.839950  4.832118  4.824132  4.815940   
96          5.039947  5.044040  5.048131  5.052215  5.056348  5.060466   
97          5.058413  5.065666  5.072850  5.079834  5.086633  5.093287   
98          5.075700  5.090641  5.106026  5.121752  5.138004  5.154803   
99          5.008033  5.007216  5.006386  5.005494  5.004638  5.003826   

parameters                      
runs         iter_18   iter_19  
0                NaN       NaN  
1                NaN       NaN  
2                NaN       NaN  
3                NaN       NaN  
4                NaN       NaN  
..               ...       ...  
95          4.807607  4.799059  
96          5.064683  5.068957  
97          5.099601  5.105629  
98          5.172241  5.190327  
99          5.003087  5.002280  

[100 rows x 100 columns]